In [ ]:
import dotenv
from langchain_openai import ChatOpenAI
import os

In [ ]:
os.environ.get("OPENAI_API_KEY")

In [ ]:
chat = ChatOpenAI(api_key=os.environ.get("OPENAI_API_KEY"), model="gpt-4o-mini")

In [ ]:
chat.invoke("什么是大模型？")

In [ ]:
os.environ.get("OPENAI_BASE_URL")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 需要注意的一点是, 这里需要指明具体的role, 在这里是system和用户
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是世界级的技术文文档编写者"),
    ("user", "{input}") # {input}为变量
])

# 我们可以把prompt和具体llm的调用和在一起。
chain = prompt | chat
message = chain.invoke({"input": "大型模型中的LangChain是什么?"})
print(message)

# print(type(message))

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

# 初始化模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是世界级的技术文档编写者。"),
    ("user", "{input}")
])

# 使用输出解析器
# output_parser = StrOutputParser() # 这行被注释了
output_parser = JsonOutputParser()

# 将其添加到链中
# chain = prompt | llm # 这行被注释了
chain = prompt | chat | output_parser

# 调用它并提出同样的问题。答案是一个字符串, 而不是ChatMessage
# chain.invoke({"input": "LangChain是什么?"}) # 这行被注释了
chain.invoke({"input": "LangChain是什么?用JSON格式回复, 问题用question, 回答用answer"})

In [ ]:
# 导入和使用 WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_path="https://www.gov.cn/xinwen/2020-06/01/content_5516649.htm",
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(id="UCAP-CONTENT"))
)
docs = loader.load()
# print(docs)

# 对于嵌入模型, 这里通过 API 调用
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 使用分割器分割文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
print(len(documents))
# 向量存储 embeddings 会将 documents 中的每个文本片段转换为向量, 并将这些向量存储在 FAISS 向量数据库中
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain_core.prompts import PromptTemplate
retriever = vector.as_retriever()
retriever.search_kwargs = {"k": 3}
docs = retriever.invoke("建设用地使用权是什么？")
# for i,doc in enumerate(docs):
# print(f"⭐第{i+1}条规定：")
# print(doc)
# 6.定义提示词模版
prompt_template = """
你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。
已知信息:
{info}
用户问：
{question}
请用中文回答用户问题。
"""
# 7.得到提示词模版对象
template = PromptTemplate.from_template(prompt_template)
# 8.得到提示词对象
prompt = template.format(info=docs, question='建设用地使用权是什么？')
## 9. 调用LLM
response = llm.invoke(prompt)
print(response.content)

In [ ]:
from langchain.tools.retriever import create_retriever_tool
# 检索器工具
retriever_tool = create_retriever_tool(
retriever, "CivilCodeRetriever",
"搜索有关中华人民共和国民法典的信息。关于中华人民共和国民法典的任何问题，您必须使用此工具!",
)
tools = [retriever_tool]
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
# https://smith.langchain.com/hub
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
# 运行代理
agent_executor.invoke({"input": "建设用地使用权是什么"})